# PyCity Schools Analysis

- My analysis
  
---

In [1]:
import pandas as pd
from pathlib import Path

# load csv files
school_data_path = Path("Resources/schools_complete.csv")
student_data_path = Path("Resources/students_complete.csv")

# assign csv files to dataframes
school_data = pd.read_csv(school_data_path)
student_data = pd.read_csv(student_data_path)

# combine data
school_data_all = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_all.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [2]:
# calculate total number of unique schools
school_count = len(school_data_all["school_name"].unique())
print(school_count)

15


In [3]:
# calculate total number of students
student_count = len(school_data_all["student_name"])
print(student_count)

39170


In [4]:
# calculate total budget
total_budget = school_data["budget"].sum()
print(total_budget)


24649428


In [5]:
# calculate average math score
average_math_score = school_data_all["math_score"].mean()
print(average_math_score)

78.98537145774827


In [6]:
# calculate average reading score
average_reading_score = school_data_all["reading_score"].mean()
print(average_reading_score)

81.87784018381414


In [7]:
# calculate the percentage of students who passed math (greater than or equal to 70)
passing_math_count = school_data_all[(school_data_all["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
print(passing_math_percentage)

74.9808526933878


In [8]:
# calculate the percentage of students who passed reading
passing_reading_count = school_data_all[(school_data_all["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
print(passing_reading_percentage)

85.80546336482001


In [9]:
# students that passed math and reading
overall_passing_count = school_data_all[
    (school_data_all["math_score"] >= 70) & (school_data_all["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = overall_passing_count / float(student_count) * 100
print(overall_passing_rate)

65.17232575950983


In [10]:
# create high-level snapshot of district's key metrics in a DataFrame
data = {'Total Schools':[school_count],'Total Students':[student_count],
        'Total Budget':[total_budget],'Average Math Score':[average_math_score],
        'Average Reading Score':[average_reading_score],'% Passing Math':[passing_math_percentage],
        '% Passing Reading':[passing_reading_percentage],'% Overall Passing':[overall_passing_rate]
        }
district_summary = pd.DataFrame(data)

# format dataframe
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

district_summary.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [11]:
# show school names
df = school_data.groupby("school_name").count().reset_index()
df[["school_name"]].head()

,school_name
0,Bailey High School
1,Cabrera High School
2,Figueroa High School
3,Ford High School
4,Griffin High School


In [121]:
# show school types
# df = school_data.groupby("type", as_index=False).count().reset_index()
# df[["type"]]
grp = school_data_all.groupby(["school_name"])
school_types = grp.first()[["type"]]
school_types = school_types.rename(columns={"type" : "School Type"})
school_types.head()

,School Type
school_name,
Bailey High School,District
Cabrera High School,Charter
Figueroa High School,District
Ford High School,District
Griffin High School,Charter


In [125]:
# total student count per school
grp = school_data_all.groupby("school_name")["student_name"].count()
per_school_counts = grp.to_frame("Student Count")
per_school_counts.head()

,Student Count
school_name,
Bailey High School,4976
Cabrera High School,1858
Figueroa High School,2949
Ford High School,2739
Griffin High School,1468


In [14]:
# total school budget
df = school_data
df["budget"] = df["budget"].map("${:,.2f}".format)
df[["school_name","budget"]].head()

,school_name,budget
0,Huang High School,"$1,910,635.00"
1,Figueroa High School,"$1,884,411.00"
2,Shelton High School,"$1,056,600.00"
3,Hernandez High School,"$3,022,020.00"
4,Griffin High School,"$917,500.00"


In [15]:
# per student budget
df = (school_data_all.groupby("school_name")["budget"].sum() /
     school_data_all.groupby("school_name")["student_name"].count()).to_frame("Per Student Budget")
df["Per Student Budget"] = df["Per Student Budget"].map("${:,.2f}".format)
df.head()

,Per Student Budget
school_name,
Bailey High School,"$3,124,928.00"
Cabrera High School,"$1,081,356.00"
Figueroa High School,"$1,884,411.00"
Ford High School,"$1,763,916.00"
Griffin High School,"$917,500.00"


In [16]:
# average math score
data = {"Average Math":[school_data_all["math_score"].mean()]}
df = pd.DataFrame(data)
df["Average Math"] = df["Average Math"].map("{:,.2f}".format)
df

,Average Math
0,78.99


In [17]:
# average reading score
data = {"Average Reading":[school_data_all["reading_score"].mean()]}
df = pd.DataFrame(data)
df["Average Reading"] = df["Average Reading"].map("{:,.2f}".format)
df

,Average Reading
0,81.88


In [18]:
# percentage of students who passed math
get_percentage = (len(student_data.loc[student_data["math_score"]>=70]) / student_count)  * 100
data = {"Passed Math":["{:,.2f}%".format(get_percentage)]}
df = pd.DataFrame(data)
df

,Passed Math
0,74.98%


In [19]:
# percentage of students who passed reading
get_percentage = (len(student_data.loc[student_data["reading_score"]>=70]) / student_count)  * 100
data = {"Passed Reading":["{:,.2f}%".format(get_percentage)]}
df = pd.DataFrame(data)
df

,Passed Reading
0,85.81%


In [20]:
# percentage of overall students who passed math and reading
get_percentage = (len(school_data_all.loc[(school_data_all["math_score"]>=70) 
& (school_data_all["reading_score"]>=70)]) / student_count)  * 100
data = {"Passed Math and Reading":["{:,.2f}%".format(get_percentage)]}
df = pd.DataFrame(data)
df

,Passed Math and Reading
0,65.17%


In [139]:
# create a school summary dataframe
dfdict = {"School Type":school_types["School Type"].squeeze(),
        "Total Students":per_school_counts["Student Count"].squeeze()}

per_school_summary = pd.DataFrame(dfdict)

per_school_summary.index.name = None
per_school_summary

,School Type,Total Students
Bailey High School,District,4976
Cabrera High School,Charter,1858
Figueroa High School,District,2949
Ford High School,District,2739
Griffin High School,Charter,1468
Hernandez High School,District,4635
Holden High School,Charter,427
Huang High School,District,2917
Johnson High School,District,4761
Pena High School,Charter,962


## Highest-Performing Schools (by % Overall Passing)

In [21]:
# highest performing schools
school_performance = school_data_all.groupby("school_name").apply(lambda x: (len(x.loc[(x["math_score"]>=70) 
                                                           & (x["reading_score"]>=70)]) 
                                                           / student_count) * 100).to_frame("Percentage").reset_index()

top_schools = school_performance.sort_values(['Percentage'],ascending=False)
top_schools["Percentage"] = top_schools["Percentage"].map("{:,.2f}%".format)
top_schools.head(5)


,school_name,Percentage
0,Bailey High School,6.94%
8,Johnson High School,6.51%
5,Hernandez High School,6.33%
10,Rodriguez High School,5.41%
13,Wilson High School,5.28%


## Lowest-Performing Schools (by % Overall Passing)

In [22]:
# lowest performing schools
school_performance = school_data_all.groupby("school_name").apply(lambda x: (len(x.loc[(x["math_score"]>=70) 
                                                           & (x["reading_score"]>=70)]) 
                                                           / student_count) * 100).to_frame("Percentage").reset_index()

bottom_schools = school_performance.sort_values(['Percentage'],ascending=True)
bottom_schools["Percentage"] = bottom_schools["Percentage"].map("{:,.2f}%".format)
bottom_schools.head(5)

,school_name,Percentage
6,Holden High School,0.97%
9,Pena High School,2.22%
4,Griffin High School,3.40%
3,Ford High School,3.80%
12,Thomas High School,3.80%


## Math Scores by Grade

In [132]:
# average students math scores by grade per school
df = school_data_all
grade_nine = df[(df["grade"] == "9th")]
grade_ten = df[(df["grade"] == "10th")]
grade_eleven = df[(df["grade"] == "11th")]
grade_twelve = df[(df["grade"] == "12th")]

scoretype = "math_score"

scores_nine = grade_nine.groupby("school_name")[scoretype].mean()
scores_ten = grade_ten.groupby("school_name")[scoretype].mean()
scores_eleven = grade_eleven.groupby("school_name")[scoretype].mean()
scores_twelve = grade_twelve.groupby("school_name")[scoretype].mean()

math_scores_by_grade = pd.DataFrame({"9th":scores_nine,"10th":scores_ten,
                                     "11th":scores_eleven,"12th":scores_twelve})

math_scores_by_grade.index.name = None
math_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Scores by Grade

In [24]:
# average students reading scores by grade per school
df = school_data_all
grade_nine = df[(df["grade"] == "9th")]
grade_ten = df[(df["grade"] == "10th")]
grade_eleven = df[(df["grade"] == "11th")]
grade_twelve = df[(df["grade"] == "12th")]

scoretype = "reading_score"

scores_nine = grade_nine.groupby("school_name")[scoretype].mean()
scores_ten = grade_ten.groupby("school_name")[scoretype].mean()
scores_eleven = grade_eleven.groupby("school_name")[scoretype].mean()
scores_twelve = grade_twelve.groupby("school_name")[scoretype].mean()

reading_scores_by_grade = pd.DataFrame({"9th":scores_nine,"10th":scores_ten,
                                     "11th":scores_eleven,"12th":scores_twelve})

reading_scores_by_grade.index.name = None
reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [ ]:
# create the bins
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [ ]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

## Scores by School Size

## Scores by School Type

## My Report

- In general, charter schools appear to perform better than district schools, as evidenced by data analysis of the top and bottom 5 schools.
- Across grades 9-12, there is no discernible difference in average math and reading scores.
- Students demonstrate consistent performance in both math and reading, as demonstrated by Bailey High School.
- Bailey High School's 10th-grade students achieve an average math score of 76.99 and an average reading score of 80.80.